# THỰC HIỆN

## 1. Thay đổi mô hình thành mô hình gợi ý
Cần lưu ý ở file demo.cc trước khi chạy: dataset - model - epoch.

Chỉnh sửa ở file demo.cc:
* Thêm thư viện time.h và 2 đối tượng time_t: start + end để đo thời gian train.
* Convolutional layer (Conv2D) có filter 5x5, stride = 1x1, padding = 0, số lượng là 6 và 16 tương ứng với mô hình.
* Pooling layer (MaxPooling2D) có kích thước là 2x2, stride = 2x2, padding = 0.

## 2. Setup cuda và cmake

Ý tưởng ban đầu là đổi file /src/layer/conv.cc thành /src/layer/conv.cu và cài đặt các filter ở đây nhưng không thể tìm ra giải pháp.

1. Thay đổi compiler của cmake để biên dịch cuda [(tham khảo)](https://dominoc925.blogspot.com/2021/03/creating-simple-cuda-with-cmake-c.html)
* Chỉnh sửa hai dòng ở file /CMakeLists.txt.
* Sửa dòng số 42 ở /third_party/eigen/Eigen/Core: math_functions.h

2. Cải tiến để chương trình có thể nhận cmd argument:
* Thêm thuộc tính public `int use_device` ở file /src/network.h và /src/layer.h
* Ý tưởng: network sẽ nhận yêu cầu sử dụng filter (GPU) từ người dùng thông qua cmd với 0 = sử dụng cpu (convolution tuần tự), k (k>0) = filter loại k. Phương thức forward của network sẽ setup `use_device` cho các layer. Giá trị `use device` chỉ được sử dụng ở file `conv.cc` để lựa chọn filter.
* Ở file `demo.cc`: setup giá trị `use_device` cho dnn (sử dụng atoi() vì cuda 12.2 không hỗ trợ stoi()).


# KẾ HOẠCH THỰC HIỆN
20-26/11: tìm hiểu về cnn, LeNet-5, setup và chạy mã nguồn mini-dnn-cpp với bộ dữ liệu mnist.

27/11 - 03/12: tìm hiểu mã nguồn mini-dnn-cpp, thay đổi mô hình với cấu trúc LeNet-5 (được tinh chỉnh theo yêu cầu của đồ án) và chạy mô hình trên tập dữ liệu fashion MNIST.

08/12 - 17/12: cài đặt GPU Convolution kernel cơ bản.



# NGUỒN THAM KHẢO
BASE MODEL: mini-dnn-cpp
* https://github.com/iamhankai/mini-dnn-cpp.git

DATASET: fashion MNIST
* https://github.com/zalandoresearch/fashion-mnist

RELATED KNOWLEDGE: CNN + LeNet-5
* https://www.kaggle.com/code/blurredmachine/lenet-architecture-a-complete-guide/notebook
* https://setosa.io/ev/image-kernels/
* https://poloclub.github.io/cnn-explainer/
* https://www.tensorflow.org/
*
